In [1]:
import pandas as pd

# Idée générale

Il s'agit dans ce notebook de mettre en place l'idée de cet [issue](https://github.com/dataforgoodfr/offseason_missiontransition_simulateur/issues/8). 

Formellement, entraîner un modèle qui à un couple `(aid, company)` est en mesure de rendre un `score` entre 0 et 1 representant les chances de succès de la demande d'aide pour savoir à quel point il est pertinent de proposer précisément cette `aid` à cette `company`.

# Préparation du dataset

## Données ADEME
Nous disposons d'un ensemble d'exemples de $n$ couples $\mathcal{S} = \{(a_i, c_i)\} _{i\leq n}$ qui ont fait l'objet d'une attribution de l'ADEME : pour tout $i\leq n$, l'entreprise $c_i$ a reçu l'aide $a_i$ proposée par l'ADEME.

Nous voulons créer un dataset labelisé, $\mathcal{D} = \{[(a_i, c_i), y_i]\}_{i\leq N}$ où $y_i$ représente les chances que l'entreprise $c_i$ obtienne (ou ait intérêt à obtenir) l'aide $a_i$.

$\mathcal{S}$ nous fournit un premier dataset avec des labels $y_i=1$, puisqu'il s'agit d'exemples où l'entreprise a effectivement obtenu l'aide en question.

Pour avoir un dataset complet, il faudrait avoir des exemples de couples $(a_i, c_i)$, avec un score $y_i$ inférieur à 1. 

## Augmentation du dataset

L'idée est donc la suivante : pour chaque aide $a_i$ dans un couple dans $\mathcal{S}$, on sait que seule l'entreprise $c_i$ a obtenu cette aide, autrement dit, tous les couples $(a_i, c_j)$ pour $j=i$ peuvent être vu comme des exemples avec un label $y_i=0$.

Ceci nous fournit donc $n(n-1)$ exemples avec $y_i=0$ ($n-1$ exemples pour les $n$ aides). Pour éviter d'avoir un dataset trop déséquilibré, on peut en garder $n$ au hasard afin de former un dataset de taille $2n$.

In [ ]:
### Dataset preparation

# Modélisation

## Features

### Côté Mission-Transition

In [3]:
mt_aids = pd.read_csv("https://dataforgood.slite.com/api/files/0wYKC93pC/data-1634140125310.csv")

In [5]:
mt_aids.head()

,id,aid_advisor_id,funder_id,ulid,source_id,name,perimeter,goal,beneficiary,aid_details,...,aid_id-2,aid_type_id,id-3,name-3,slug-2,aid_id-3,environmental_topic_id,id-4,name-4,slug-3
0,198,NaN,10.0,017c7a50-c5ea-5d3b-e8bc-569be89a32fd,at_94683,Financer les études et tests préalables aux in...,NATIONAL,NaN,NaN,<p>\n <strong>\n Aide - « Études et tests pré...,...,198.0,1.0,NaN,NaN,NaN,198.0,1.0,NaN,NaN,NaN
1,187,NaN,10.0,017c7a50-c4c6-704e-7235-0b41ca6f711e,at_58447,Soutenir financièrement le développement de l'...,NATIONAL,NaN,NaN,<p>\n Le dispositif de l'ADEME – ORPLAST (Obje...,...,187.0,1.0,NaN,NaN,NaN,187.0,1.0,NaN,NaN,NaN
2,13,NaN,3.0,017c7a50-b75f-c83d-01bc-49b7055c407f,at_90955,Soutenir financièrement l'intégration de matiè...,REGIONAL,NaN,NaN,<p>\n <p>\n <p>\n <p>\n <p>\n ORPLAS...,...,13.0,1.0,NaN,NaN,NaN,13.0,1.0,13.0,Stockage de l'énergie,stockage-de-lenergie
3,13,NaN,3.0,017c7a50-b75f-c83d-01bc-49b7055c407f,at_90955,Soutenir financièrement l'intégration de matiè...,REGIONAL,NaN,NaN,<p>\n <p>\n <p>\n <p>\n <p>\n ORPLAS...,...,13.0,1.0,NaN,NaN,NaN,13.0,2.0,13.0,Stockage de l'énergie,stockage-de-lenergie
4,213,NaN,10.0,017c7a50-c7a5-fc2f-8bb3-ca4b562a0715,at_94687,Financer les études de préfiguration d'une dém...,NATIONAL,NaN,NaN,<p>\n Aide - « Étude de préfiguration d'une dé...,...,213.0,1.0,NaN,NaN,NaN,213.0,3.0,NaN,NaN,NaN


In [8]:
mt_aids.sample(1).iloc[0]

id                                                                           158
aid_advisor_id                                                               NaN
funder_id                                                                   10.0
ulid                                        017c7a50-c1b8-d871-f4a1-879baffc77fa
source_id                                                               at_94223
name                           Accompagner les projets des entreprises engagé...
perimeter                                                               NATIONAL
goal                                                                         NaN
beneficiary                                                                  NaN
aid_details                    <p>\n <strong>\n  AAP - « Investissements d'éc...
eligibility                    <p>\n <strong>\n  Critères d'éligibilité :\n <...
conditions                                                                   NaN
funding_source_url          

Natural language features :
- name
- aid_details
- eligibility

Numeric features :
- subvention_rate_upper_bound
- subvention_rate_lower_bound

Categorical features :
- funding_types

## Côté ADEME

Côté ADEME, on ne trouvera pas plus de détail sur les aides, en revanche on obtient le nom de l'entreprise, ce qui nous permet d'obtenir les informations récupérables par l'API SIRENE comme  [ici](https://github.com/dataforgoodfr/offseason_missiontransition_simulateur/issues/4).

## Modèle

Les features en langage naturel peuvent être traités avec [SBERT](https://www.sbert.net/docs/quickstart.html) pour ensuite entraîner un modèle de classification.